In [1]:
# -*- coding: utf8 -*-
import urllib
from urllib.request import urlopen
import requests
import json
from bs4 import BeautifulSoup
from pprint import pprint
import random
from requests import Session
import time
import csv
import random
import http.cookiejar as cookielib

In [2]:
food_type = '蛋制品'
check_flag = '不合格'
file_name = food_type+'-'+check_flag+'.csv'
if check_flag == '合格':
    order_by = 1
else:
    order_by = 0
    
info = {}
info['food_type'] = food_type.encode('UTF-8')
info['check_flag'] = check_flag.encode('UTF-8')
info = urllib.parse.urlencode(info)
info = info.replace('&', '%22%2C%22').replace('=','%22%3A%22')+'%22%2C%22'    


total_info = []
session = requests.Session()
session.cookies = cookielib.LWPCookieJar(filename='sacspider')
try:
    session.cookies.load(ignore_discard=True)
except:
    print('cookie 文件未能加载')

    
url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
headers = {
        'Accept':'application/json',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        'Host':'www.foods12331.cn',
        'Origin':'http://www.foods12331.cn',
        'proxy-connection': "keep-alive",
        'cache-control': "no-cache",
        'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
    }
    
payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22'.format(order_by)+'pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(0)

response = session.post(url=url, data=payload, headers=headers)
total_food = response.json()['resultData']['total']
        
print(food_type+'-'+check_flag+'共有{}个食品结果'.format(total_food))
food_page = total_food // 20
print('开始抓取数据.....')

for i in range(0,food_page,1):
#     print(i)
    url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
    headers = {
        'Accept':'application/json',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        'Host':'www.foods12331.cn',
        'Origin':'http://www.foods12331.cn',
        'proxy-connection': "keep-alive",
        'cache-control': "no-cache",
        'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
    }
#     payload = "filters=%7B%22food_type%22%3A%22%E9%A5%AE%E6%96%99%22%2C%22check_flag%22%3A%22%E5%90%88%E6%A0%BC%22%2C%22order_by%22%3A%221%22%2C%22pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D".format(i)
    

    payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22'.format(order_by)+'pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(i)

    response = session.post(url=url, data=payload, headers=headers)
    if response.status_code // 100 != 2:
        print('[INFO ERROR] 第'+str(i)+'页内容无法爬取')
        continue
    for ele in response.json()['resultData']['items']:
        params = {}
        food_id = ele['id']
        food_name = ele['food_name']
        production_name = ele['production_name']
        check_num = ele['check_num']
        qualified_num = ele['qualified_num']
        unqualified_num = ele['unqualified_num']
        params['food_name'] = ele['food_name'].encode('UTF-8')
        params['production_name'] = ele['production_name'].encode('UTF-8')
        params['food_model'] = ele['food_model'].encode('UTF-8')
        payload = urllib.parse.urlencode(params)
        url = 'http://www.foods12331.cn/food/detail/getResult.json'
        headers = {
            'Accept':'*/*',
            'Accept-Encoding':'gzip, deflate',
            'Accept-Language':'zh-CN,zh;q=0.8',
            'Connection':'keep-alive',
            'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
            'Host':'www.foods12331.cn',
            'Origin':'http://www.foods12331.cn',
            'proxy-connection': "keep-alive",
            'cache-control': "no-cache",
            'Referer':'http://www.foods12331.cn/web/list.jsp?foodId={}&food_type=%E9%A5%AE%E6%96%99&food_condition='.format(food_id),
            'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'X-Requested-With':'XMLHttpRequest'
        }
        sleep_time = random.randint(2,4)
        time.sleep(sleep_time)
        response = session.post(url=url, data=payload, headers=headers)
        if response.status_code // 100 != 2:
            print('[INFO ERROR] food_id为'+food_id+'的食品无法抓取')
            continue
        for ele in response.json()['resultData']['foods']:
            food_dict = {}
            production_address = ele['production_adress']
            food_product_time = ele['food_product_time']
            food_model = ele['food_model']
            check_flag = ele['check_flag']
            if order_by == 0:
                if ele['unqualified_reason']:
                    check_flag = check_flag + " " + ele['unqualified_reason']
            sampling_name = ele['sampling_name']
            sampling_province = ele['sampling_province']
            check_projiect = ele['check_projiect']

            food_dict = {
                'food_name':food_name,
                'production_name':production_name,
                'check_num':check_num,
                'qualified_num':qualified_num,
                'unqualified_num':unqualified_num,
                'production_address':production_address,
                'food_product_time':food_product_time,
                'food_model':food_model,
                'check_flag':check_flag,
                'sampling_name':sampling_name,
                'sampling_province':sampling_province,
                'check_projiect':check_projiect
                }
            total_info.append(food_dict)
#             print(food_dict)
#         sleep_time = random.randint(1, 4)
#         time.sleep(sleep_time)
    session.cookies.save()
    print(u'已完成'+ str((i+1)*20) +u'个食品的数据抓取')


# 最后一页page_size不是20时的数据抓取
remain_page_size = total_food % 20
remain_page = total_food // 20

print('抓取最后{}个食品的数据......'.format(remain_page_size))

url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
headers = {
    'Accept':'application/json',
    'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.8',
    'Connection':'keep-alive',
    'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
    'Host':'www.foods12331.cn',
    'Origin':'http://www.foods12331.cn',
    'proxy-connection': "keep-alive",
    'cache-control': "no-cache",
    'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'X-Requested-With':'XMLHttpRequest'
}


payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22pageNo%22%3A{}%2C%22pageSize%22%3A{}%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(order_by,remain_page,remain_page_size)

response = session.post(url=url, data=payload, headers=headers)

flag = True

if response.status_code // 100 != 2:
    print('[INFO ERROR] 第'+str(i)+'页内容无法爬取')
    flag = False
if flag:
    for ele in response.json()['resultData']['items']:
        params = {}
        food_id = ele['id']
        food_name = ele['food_name']
        production_name = ele['production_name']
        check_num = ele['check_num']
        qualified_num = ele['qualified_num']
        unqualified_num = ele['unqualified_num']
        params['food_name'] = ele['food_name'].encode('UTF-8')
        params['production_name'] = ele['production_name'].encode('UTF-8')
        params['food_model'] = ele['food_model'].encode('UTF-8')
        payload = urllib.parse.urlencode(params)
        url = 'http://www.foods12331.cn/food/detail/getResult.json'
        headers = {
                'Accept':'*/*',
                'Accept-Encoding':'gzip, deflate',
                'Accept-Language':'zh-CN,zh;q=0.8',
                'Connection':'keep-alive',
                'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
                'Host':'www.foods12331.cn',
                'Origin':'http://www.foods12331.cn',
                'proxy-connection': "keep-alive",
                'cache-control': "no-cache",
                'Referer':'http://www.foods12331.cn/web/list.jsp?foodId={}&food_type=%E9%A5%AE%E6%96%99&food_condition='.format(food_id),
                'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
                'X-Requested-With':'XMLHttpRequest'
            }
        time.sleep(2)
        response = session.post(url=url, data=payload, headers=headers)
        if response.status_code // 100 != 2:
            print('[INFO ERROR] food_id为'+food_id+'的食品无法抓取')
            continue
        for ele in response.json()['resultData']['foods']:
            food_dict = {}
            production_address = ele['production_adress']
            food_product_time = ele['food_product_time']
            food_model = ele['food_model']
            check_flag = ele['check_flag']
            if order_by == 0:
                if ele['unqualified_reason']:
                    check_flag = check_flag + " " + ele['unqualified_reason']
            sampling_name = ele['sampling_name']
            sampling_province = ele['sampling_province']
            check_projiect = ele['check_projiect']
            food_dict = {
                'food_name':food_name,
                'production_name':production_name,
                'check_num':check_num,
                'qualified_num':qualified_num,
                'unqualified_num':unqualified_num,
                'production_address':production_address,
                'food_product_time':food_product_time,
                'food_model':food_model,
                'check_flag':check_flag,
                'sampling_name':sampling_name,
                'sampling_province':sampling_province,
                'check_projiect':check_projiect
                }
            total_info.append(food_dict)
    
print('数据抓取完成!\n开始写入数据.....')
    
headers = ['food_name', 'production_name', 'check_num', 'qualified_num', 'unqualified_num', 'production_address', 'food_product_time', 'food_model', 'check_flag', 'sampling_name', 'sampling_province', 'check_projiect']

with open(file_name, 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(total_info)
print('数据写入完成!')

蛋制品-不合格共有44个食品结果
开始抓取数据.....
已完成20个食品的数据抓取
[INFO ERROR] food_id为65bc7f34-885e-457b-8c55-3992a7fe528f的食品无法抓取
已完成40个食品的数据抓取
抓取最后4个食品的数据......
数据抓取完成!
开始写入数据.....
数据写入完成!


In [3]:
food_type = '水产制品'
check_flag = '不合格'
file_name = food_type+'-'+check_flag+'.csv'
if check_flag == '合格':
    order_by = 1
else:
    order_by = 0
    
info = {}
info['food_type'] = food_type.encode('UTF-8')
info['check_flag'] = check_flag.encode('UTF-8')
info = urllib.parse.urlencode(info)
info = info.replace('&', '%22%2C%22').replace('=','%22%3A%22')+'%22%2C%22'    


total_info = []
session = requests.Session()
session.cookies = cookielib.LWPCookieJar(filename='sacspider')
try:
    session.cookies.load(ignore_discard=True)
except:
    print('cookie 文件未能加载')

    
url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
headers = {
        'Accept':'application/json',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        'Host':'www.foods12331.cn',
        'Origin':'http://www.foods12331.cn',
        'proxy-connection': "keep-alive",
        'cache-control': "no-cache",
        'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
    }
    
payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22'.format(order_by)+'pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(0)

response = session.post(url=url, data=payload, headers=headers)
total_food = response.json()['resultData']['total']
        
print(food_type+'-'+check_flag+'共有{}个食品结果'.format(total_food))
food_page = total_food // 20
print('开始抓取数据.....')

for i in range(0,food_page,1):
#     print(i)
    url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
    headers = {
        'Accept':'application/json',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        'Host':'www.foods12331.cn',
        'Origin':'http://www.foods12331.cn',
        'proxy-connection': "keep-alive",
        'cache-control': "no-cache",
        'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
    }
#     payload = "filters=%7B%22food_type%22%3A%22%E9%A5%AE%E6%96%99%22%2C%22check_flag%22%3A%22%E5%90%88%E6%A0%BC%22%2C%22order_by%22%3A%221%22%2C%22pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D".format(i)
    

    payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22'.format(order_by)+'pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(i)

    response = session.post(url=url, data=payload, headers=headers)
    if response.status_code // 100 != 2:
        print('[INFO ERROR] 第'+str(i)+'页内容无法爬取')
        continue
    for ele in response.json()['resultData']['items']:
        params = {}
        food_id = ele['id']
        food_name = ele['food_name']
        production_name = ele['production_name']
        check_num = ele['check_num']
        qualified_num = ele['qualified_num']
        unqualified_num = ele['unqualified_num']
        params['food_name'] = ele['food_name'].encode('UTF-8')
        params['production_name'] = ele['production_name'].encode('UTF-8')
        params['food_model'] = ele['food_model'].encode('UTF-8')
        payload = urllib.parse.urlencode(params)
        url = 'http://www.foods12331.cn/food/detail/getResult.json'
        headers = {
            'Accept':'*/*',
            'Accept-Encoding':'gzip, deflate',
            'Accept-Language':'zh-CN,zh;q=0.8',
            'Connection':'keep-alive',
            'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
            'Host':'www.foods12331.cn',
            'Origin':'http://www.foods12331.cn',
            'proxy-connection': "keep-alive",
            'cache-control': "no-cache",
            'Referer':'http://www.foods12331.cn/web/list.jsp?foodId={}&food_type=%E9%A5%AE%E6%96%99&food_condition='.format(food_id),
            'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'X-Requested-With':'XMLHttpRequest'
        }
        sleep_time = random.randint(2,4)
        time.sleep(sleep_time)
        response = session.post(url=url, data=payload, headers=headers)
        if response.status_code // 100 != 2:
            print('[INFO ERROR] food_id为'+food_id+'的食品无法抓取')
            continue
        for ele in response.json()['resultData']['foods']:
            food_dict = {}
            production_address = ele['production_adress']
            food_product_time = ele['food_product_time']
            food_model = ele['food_model']
            check_flag = ele['check_flag']
            if order_by == 0:
                if ele['unqualified_reason']:
                    check_flag = check_flag + " " + ele['unqualified_reason']
            sampling_name = ele['sampling_name']
            sampling_province = ele['sampling_province']
            check_projiect = ele['check_projiect']

            food_dict = {
                'food_name':food_name,
                'production_name':production_name,
                'check_num':check_num,
                'qualified_num':qualified_num,
                'unqualified_num':unqualified_num,
                'production_address':production_address,
                'food_product_time':food_product_time,
                'food_model':food_model,
                'check_flag':check_flag,
                'sampling_name':sampling_name,
                'sampling_province':sampling_province,
                'check_projiect':check_projiect
                }
            total_info.append(food_dict)
#             print(food_dict)
#         sleep_time = random.randint(1, 4)
#         time.sleep(sleep_time)
    session.cookies.save()
    print(u'已完成'+ str((i+1)*20) +u'个食品的数据抓取')


# 最后一页page_size不是20时的数据抓取
remain_page_size = total_food % 20
remain_page = total_food // 20

print('抓取最后{}个食品的数据......'.format(remain_page_size))

url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
headers = {
    'Accept':'application/json',
    'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.8',
    'Connection':'keep-alive',
    'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
    'Host':'www.foods12331.cn',
    'Origin':'http://www.foods12331.cn',
    'proxy-connection': "keep-alive",
    'cache-control': "no-cache",
    'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'X-Requested-With':'XMLHttpRequest'
}


payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22pageNo%22%3A{}%2C%22pageSize%22%3A{}%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(order_by,remain_page,remain_page_size)

response = session.post(url=url, data=payload, headers=headers)

flag = True

if response.status_code // 100 != 2:
    print('[INFO ERROR] 第'+str(i)+'页内容无法爬取')
    flag = False
if flag:
    for ele in response.json()['resultData']['items']:
        params = {}
        food_id = ele['id']
        food_name = ele['food_name']
        production_name = ele['production_name']
        check_num = ele['check_num']
        qualified_num = ele['qualified_num']
        unqualified_num = ele['unqualified_num']
        params['food_name'] = ele['food_name'].encode('UTF-8')
        params['production_name'] = ele['production_name'].encode('UTF-8')
        params['food_model'] = ele['food_model'].encode('UTF-8')
        payload = urllib.parse.urlencode(params)
        url = 'http://www.foods12331.cn/food/detail/getResult.json'
        headers = {
                'Accept':'*/*',
                'Accept-Encoding':'gzip, deflate',
                'Accept-Language':'zh-CN,zh;q=0.8',
                'Connection':'keep-alive',
                'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
                'Host':'www.foods12331.cn',
                'Origin':'http://www.foods12331.cn',
                'proxy-connection': "keep-alive",
                'cache-control': "no-cache",
                'Referer':'http://www.foods12331.cn/web/list.jsp?foodId={}&food_type=%E9%A5%AE%E6%96%99&food_condition='.format(food_id),
                'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
                'X-Requested-With':'XMLHttpRequest'
            }
        time.sleep(2)
        response = session.post(url=url, data=payload, headers=headers)
        if response.status_code // 100 != 2:
            print('[INFO ERROR] food_id为'+food_id+'的食品无法抓取')
            continue
        for ele in response.json()['resultData']['foods']:
            food_dict = {}
            production_address = ele['production_adress']
            food_product_time = ele['food_product_time']
            food_model = ele['food_model']
            check_flag = ele['check_flag']
            if order_by == 0:
                if ele['unqualified_reason']:
                    check_flag = check_flag + " " + ele['unqualified_reason']
            sampling_name = ele['sampling_name']
            sampling_province = ele['sampling_province']
            check_projiect = ele['check_projiect']
            food_dict = {
                'food_name':food_name,
                'production_name':production_name,
                'check_num':check_num,
                'qualified_num':qualified_num,
                'unqualified_num':unqualified_num,
                'production_address':production_address,
                'food_product_time':food_product_time,
                'food_model':food_model,
                'check_flag':check_flag,
                'sampling_name':sampling_name,
                'sampling_province':sampling_province,
                'check_projiect':check_projiect
                }
            total_info.append(food_dict)
    
print('数据抓取完成!\n开始写入数据.....')
    
headers = ['food_name', 'production_name', 'check_num', 'qualified_num', 'unqualified_num', 'production_address', 'food_product_time', 'food_model', 'check_flag', 'sampling_name', 'sampling_province', 'check_projiect']

with open(file_name, 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(total_info)
print('数据写入完成!')

水产制品-不合格共有599个食品结果
开始抓取数据.....
已完成20个食品的数据抓取
已完成40个食品的数据抓取
已完成60个食品的数据抓取
已完成80个食品的数据抓取
已完成100个食品的数据抓取
已完成120个食品的数据抓取
已完成140个食品的数据抓取
已完成160个食品的数据抓取
已完成180个食品的数据抓取
已完成200个食品的数据抓取
已完成220个食品的数据抓取
已完成240个食品的数据抓取
已完成260个食品的数据抓取
已完成280个食品的数据抓取
已完成300个食品的数据抓取
已完成320个食品的数据抓取
已完成340个食品的数据抓取
已完成360个食品的数据抓取
已完成380个食品的数据抓取
已完成400个食品的数据抓取
已完成420个食品的数据抓取
已完成440个食品的数据抓取
已完成460个食品的数据抓取
已完成480个食品的数据抓取
已完成500个食品的数据抓取
已完成520个食品的数据抓取
已完成540个食品的数据抓取
已完成560个食品的数据抓取
已完成580个食品的数据抓取
抓取最后19个食品的数据......
数据抓取完成!
开始写入数据.....
数据写入完成!


In [4]:
food_type = '豆制品'
check_flag = '不合格'
file_name = food_type+'-'+check_flag+'.csv'
if check_flag == '合格':
    order_by = 1
else:
    order_by = 0
    
info = {}
info['food_type'] = food_type.encode('UTF-8')
info['check_flag'] = check_flag.encode('UTF-8')
info = urllib.parse.urlencode(info)
info = info.replace('&', '%22%2C%22').replace('=','%22%3A%22')+'%22%2C%22'    


total_info = []
session = requests.Session()
session.cookies = cookielib.LWPCookieJar(filename='sacspider')
try:
    session.cookies.load(ignore_discard=True)
except:
    print('cookie 文件未能加载')

    
url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
headers = {
        'Accept':'application/json',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        'Host':'www.foods12331.cn',
        'Origin':'http://www.foods12331.cn',
        'proxy-connection': "keep-alive",
        'cache-control': "no-cache",
        'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
    }
    
payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22'.format(order_by)+'pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(0)

response = session.post(url=url, data=payload, headers=headers)
total_food = response.json()['resultData']['total']
        
print(food_type+'-'+check_flag+'共有{}个食品结果'.format(total_food))
food_page = total_food // 20
print('开始抓取数据.....')

for i in range(0,food_page,1):
#     print(i)
    url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
    headers = {
        'Accept':'application/json',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        'Host':'www.foods12331.cn',
        'Origin':'http://www.foods12331.cn',
        'proxy-connection': "keep-alive",
        'cache-control': "no-cache",
        'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
    }
#     payload = "filters=%7B%22food_type%22%3A%22%E9%A5%AE%E6%96%99%22%2C%22check_flag%22%3A%22%E5%90%88%E6%A0%BC%22%2C%22order_by%22%3A%221%22%2C%22pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D".format(i)
    

    payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22'.format(order_by)+'pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(i)

    response = session.post(url=url, data=payload, headers=headers)
    if response.status_code // 100 != 2:
        print('[INFO ERROR] 第'+str(i)+'页内容无法爬取')
        continue
    for ele in response.json()['resultData']['items']:
        params = {}
        food_id = ele['id']
        food_name = ele['food_name']
        production_name = ele['production_name']
        check_num = ele['check_num']
        qualified_num = ele['qualified_num']
        unqualified_num = ele['unqualified_num']
        params['food_name'] = ele['food_name'].encode('UTF-8')
        params['production_name'] = ele['production_name'].encode('UTF-8')
        params['food_model'] = ele['food_model'].encode('UTF-8')
        payload = urllib.parse.urlencode(params)
        url = 'http://www.foods12331.cn/food/detail/getResult.json'
        headers = {
            'Accept':'*/*',
            'Accept-Encoding':'gzip, deflate',
            'Accept-Language':'zh-CN,zh;q=0.8',
            'Connection':'keep-alive',
            'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
            'Host':'www.foods12331.cn',
            'Origin':'http://www.foods12331.cn',
            'proxy-connection': "keep-alive",
            'cache-control': "no-cache",
            'Referer':'http://www.foods12331.cn/web/list.jsp?foodId={}&food_type=%E9%A5%AE%E6%96%99&food_condition='.format(food_id),
            'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'X-Requested-With':'XMLHttpRequest'
        }
        sleep_time = random.randint(2,4)
        time.sleep(sleep_time)
        response = session.post(url=url, data=payload, headers=headers)
        if response.status_code // 100 != 2:
            print('[INFO ERROR] food_id为'+food_id+'的食品无法抓取')
            continue
        for ele in response.json()['resultData']['foods']:
            food_dict = {}
            production_address = ele['production_adress']
            food_product_time = ele['food_product_time']
            food_model = ele['food_model']
            check_flag = ele['check_flag']
            if order_by == 0:
                if ele['unqualified_reason']:
                    check_flag = check_flag + " " + ele['unqualified_reason']
            sampling_name = ele['sampling_name']
            sampling_province = ele['sampling_province']
            check_projiect = ele['check_projiect']

            food_dict = {
                'food_name':food_name,
                'production_name':production_name,
                'check_num':check_num,
                'qualified_num':qualified_num,
                'unqualified_num':unqualified_num,
                'production_address':production_address,
                'food_product_time':food_product_time,
                'food_model':food_model,
                'check_flag':check_flag,
                'sampling_name':sampling_name,
                'sampling_province':sampling_province,
                'check_projiect':check_projiect
                }
            total_info.append(food_dict)
#             print(food_dict)
#         sleep_time = random.randint(1, 4)
#         time.sleep(sleep_time)
    session.cookies.save()
    print(u'已完成'+ str((i+1)*20) +u'个食品的数据抓取')


# 最后一页page_size不是20时的数据抓取
remain_page_size = total_food % 20
remain_page = total_food // 20

print('抓取最后{}个食品的数据......'.format(remain_page_size))

url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
headers = {
    'Accept':'application/json',
    'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.8',
    'Connection':'keep-alive',
    'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
    'Host':'www.foods12331.cn',
    'Origin':'http://www.foods12331.cn',
    'proxy-connection': "keep-alive",
    'cache-control': "no-cache",
    'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'X-Requested-With':'XMLHttpRequest'
}


payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22pageNo%22%3A{}%2C%22pageSize%22%3A{}%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(order_by,remain_page,remain_page_size)

response = session.post(url=url, data=payload, headers=headers)

flag = True

if response.status_code // 100 != 2:
    print('[INFO ERROR] 第'+str(i)+'页内容无法爬取')
    flag = False
if flag:
    for ele in response.json()['resultData']['items']:
        params = {}
        food_id = ele['id']
        food_name = ele['food_name']
        production_name = ele['production_name']
        check_num = ele['check_num']
        qualified_num = ele['qualified_num']
        unqualified_num = ele['unqualified_num']
        params['food_name'] = ele['food_name'].encode('UTF-8')
        params['production_name'] = ele['production_name'].encode('UTF-8')
        params['food_model'] = ele['food_model'].encode('UTF-8')
        payload = urllib.parse.urlencode(params)
        url = 'http://www.foods12331.cn/food/detail/getResult.json'
        headers = {
                'Accept':'*/*',
                'Accept-Encoding':'gzip, deflate',
                'Accept-Language':'zh-CN,zh;q=0.8',
                'Connection':'keep-alive',
                'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
                'Host':'www.foods12331.cn',
                'Origin':'http://www.foods12331.cn',
                'proxy-connection': "keep-alive",
                'cache-control': "no-cache",
                'Referer':'http://www.foods12331.cn/web/list.jsp?foodId={}&food_type=%E9%A5%AE%E6%96%99&food_condition='.format(food_id),
                'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
                'X-Requested-With':'XMLHttpRequest'
            }
        time.sleep(2)
        response = session.post(url=url, data=payload, headers=headers)
        if response.status_code // 100 != 2:
            print('[INFO ERROR] food_id为'+food_id+'的食品无法抓取')
            continue
        for ele in response.json()['resultData']['foods']:
            food_dict = {}
            production_address = ele['production_adress']
            food_product_time = ele['food_product_time']
            food_model = ele['food_model']
            check_flag = ele['check_flag']
            if order_by == 0:
                if ele['unqualified_reason']:
                    check_flag = check_flag + " " + ele['unqualified_reason']
            sampling_name = ele['sampling_name']
            sampling_province = ele['sampling_province']
            check_projiect = ele['check_projiect']
            food_dict = {
                'food_name':food_name,
                'production_name':production_name,
                'check_num':check_num,
                'qualified_num':qualified_num,
                'unqualified_num':unqualified_num,
                'production_address':production_address,
                'food_product_time':food_product_time,
                'food_model':food_model,
                'check_flag':check_flag,
                'sampling_name':sampling_name,
                'sampling_province':sampling_province,
                'check_projiect':check_projiect
                }
            total_info.append(food_dict)
    
print('数据抓取完成!\n开始写入数据.....')
    
headers = ['food_name', 'production_name', 'check_num', 'qualified_num', 'unqualified_num', 'production_address', 'food_product_time', 'food_model', 'check_flag', 'sampling_name', 'sampling_province', 'check_projiect']

with open(file_name, 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(total_info)
print('数据写入完成!')

豆制品-不合格共有242个食品结果
开始抓取数据.....
已完成20个食品的数据抓取
已完成40个食品的数据抓取
已完成60个食品的数据抓取
已完成80个食品的数据抓取
已完成100个食品的数据抓取
已完成120个食品的数据抓取
已完成140个食品的数据抓取
已完成160个食品的数据抓取
已完成180个食品的数据抓取
已完成200个食品的数据抓取
已完成220个食品的数据抓取
已完成240个食品的数据抓取
抓取最后2个食品的数据......
数据抓取完成!
开始写入数据.....
数据写入完成!


In [5]:
food_type = '糕点'
check_flag = '不合格'
file_name = food_type+'-'+check_flag+'.csv'
if check_flag == '合格':
    order_by = 1
else:
    order_by = 0
    
info = {}
info['food_type'] = food_type.encode('UTF-8')
info['check_flag'] = check_flag.encode('UTF-8')
info = urllib.parse.urlencode(info)
info = info.replace('&', '%22%2C%22').replace('=','%22%3A%22')+'%22%2C%22'    


total_info = []
session = requests.Session()
session.cookies = cookielib.LWPCookieJar(filename='sacspider')
try:
    session.cookies.load(ignore_discard=True)
except:
    print('cookie 文件未能加载')

    
url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
headers = {
        'Accept':'application/json',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        'Host':'www.foods12331.cn',
        'Origin':'http://www.foods12331.cn',
        'proxy-connection': "keep-alive",
        'cache-control': "no-cache",
        'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
    }
    
payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22'.format(order_by)+'pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(0)

response = session.post(url=url, data=payload, headers=headers)
total_food = response.json()['resultData']['total']
        
print(food_type+'-'+check_flag+'共有{}个食品结果'.format(total_food))
food_page = total_food // 20
print('开始抓取数据.....')

for i in range(0,food_page,1):
#     print(i)
    url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
    headers = {
        'Accept':'application/json',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        'Host':'www.foods12331.cn',
        'Origin':'http://www.foods12331.cn',
        'proxy-connection': "keep-alive",
        'cache-control': "no-cache",
        'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
    }
#     payload = "filters=%7B%22food_type%22%3A%22%E9%A5%AE%E6%96%99%22%2C%22check_flag%22%3A%22%E5%90%88%E6%A0%BC%22%2C%22order_by%22%3A%221%22%2C%22pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D".format(i)
    

    payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22'.format(order_by)+'pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(i)

    response = session.post(url=url, data=payload, headers=headers)
    if response.status_code // 100 != 2:
        print('[INFO ERROR] 第'+str(i)+'页内容无法爬取')
        continue
    for ele in response.json()['resultData']['items']:
        params = {}
        food_id = ele['id']
        food_name = ele['food_name']
        production_name = ele['production_name']
        check_num = ele['check_num']
        qualified_num = ele['qualified_num']
        unqualified_num = ele['unqualified_num']
        params['food_name'] = ele['food_name'].encode('UTF-8')
        params['production_name'] = ele['production_name'].encode('UTF-8')
        params['food_model'] = ele['food_model'].encode('UTF-8')
        payload = urllib.parse.urlencode(params)
        url = 'http://www.foods12331.cn/food/detail/getResult.json'
        headers = {
            'Accept':'*/*',
            'Accept-Encoding':'gzip, deflate',
            'Accept-Language':'zh-CN,zh;q=0.8',
            'Connection':'keep-alive',
            'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
            'Host':'www.foods12331.cn',
            'Origin':'http://www.foods12331.cn',
            'proxy-connection': "keep-alive",
            'cache-control': "no-cache",
            'Referer':'http://www.foods12331.cn/web/list.jsp?foodId={}&food_type=%E9%A5%AE%E6%96%99&food_condition='.format(food_id),
            'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'X-Requested-With':'XMLHttpRequest'
        }
        sleep_time = random.randint(2,4)
        time.sleep(sleep_time)
        response = session.post(url=url, data=payload, headers=headers)
        if response.status_code // 100 != 2:
            print('[INFO ERROR] food_id为'+food_id+'的食品无法抓取')
            continue
        for ele in response.json()['resultData']['foods']:
            food_dict = {}
            production_address = ele['production_adress']
            food_product_time = ele['food_product_time']
            food_model = ele['food_model']
            check_flag = ele['check_flag']
            if order_by == 0:
                if ele['unqualified_reason']:
                    check_flag = check_flag + " " + ele['unqualified_reason']
            sampling_name = ele['sampling_name']
            sampling_province = ele['sampling_province']
            check_projiect = ele['check_projiect']

            food_dict = {
                'food_name':food_name,
                'production_name':production_name,
                'check_num':check_num,
                'qualified_num':qualified_num,
                'unqualified_num':unqualified_num,
                'production_address':production_address,
                'food_product_time':food_product_time,
                'food_model':food_model,
                'check_flag':check_flag,
                'sampling_name':sampling_name,
                'sampling_province':sampling_province,
                'check_projiect':check_projiect
                }
            total_info.append(food_dict)
#             print(food_dict)
#         sleep_time = random.randint(1, 4)
#         time.sleep(sleep_time)
    session.cookies.save()
    print(u'已完成'+ str((i+1)*20) +u'个食品的数据抓取')


# 最后一页page_size不是20时的数据抓取
remain_page_size = total_food % 20
remain_page = total_food // 20

print('抓取最后{}个食品的数据......'.format(remain_page_size))

url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
headers = {
    'Accept':'application/json',
    'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.8',
    'Connection':'keep-alive',
    'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
    'Host':'www.foods12331.cn',
    'Origin':'http://www.foods12331.cn',
    'proxy-connection': "keep-alive",
    'cache-control': "no-cache",
    'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'X-Requested-With':'XMLHttpRequest'
}


payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22pageNo%22%3A{}%2C%22pageSize%22%3A{}%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(order_by,remain_page,remain_page_size)

response = session.post(url=url, data=payload, headers=headers)

flag = True

if response.status_code // 100 != 2:
    print('[INFO ERROR] 第'+str(i)+'页内容无法爬取')
    flag = False
if flag:
    for ele in response.json()['resultData']['items']:
        params = {}
        food_id = ele['id']
        food_name = ele['food_name']
        production_name = ele['production_name']
        check_num = ele['check_num']
        qualified_num = ele['qualified_num']
        unqualified_num = ele['unqualified_num']
        params['food_name'] = ele['food_name'].encode('UTF-8')
        params['production_name'] = ele['production_name'].encode('UTF-8')
        params['food_model'] = ele['food_model'].encode('UTF-8')
        payload = urllib.parse.urlencode(params)
        url = 'http://www.foods12331.cn/food/detail/getResult.json'
        headers = {
                'Accept':'*/*',
                'Accept-Encoding':'gzip, deflate',
                'Accept-Language':'zh-CN,zh;q=0.8',
                'Connection':'keep-alive',
                'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
                'Host':'www.foods12331.cn',
                'Origin':'http://www.foods12331.cn',
                'proxy-connection': "keep-alive",
                'cache-control': "no-cache",
                'Referer':'http://www.foods12331.cn/web/list.jsp?foodId={}&food_type=%E9%A5%AE%E6%96%99&food_condition='.format(food_id),
                'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
                'X-Requested-With':'XMLHttpRequest'
            }
        time.sleep(2)
        response = session.post(url=url, data=payload, headers=headers)
        if response.status_code // 100 != 2:
            print('[INFO ERROR] food_id为'+food_id+'的食品无法抓取')
            continue
        for ele in response.json()['resultData']['foods']:
            food_dict = {}
            production_address = ele['production_adress']
            food_product_time = ele['food_product_time']
            food_model = ele['food_model']
            check_flag = ele['check_flag']
            if order_by == 0:
                if ele['unqualified_reason']:
                    check_flag = check_flag + " " + ele['unqualified_reason']
            sampling_name = ele['sampling_name']
            sampling_province = ele['sampling_province']
            check_projiect = ele['check_projiect']
            food_dict = {
                'food_name':food_name,
                'production_name':production_name,
                'check_num':check_num,
                'qualified_num':qualified_num,
                'unqualified_num':unqualified_num,
                'production_address':production_address,
                'food_product_time':food_product_time,
                'food_model':food_model,
                'check_flag':check_flag,
                'sampling_name':sampling_name,
                'sampling_province':sampling_province,
                'check_projiect':check_projiect
                }
            total_info.append(food_dict)
    
print('数据抓取完成!\n开始写入数据.....')
    
headers = ['food_name', 'production_name', 'check_num', 'qualified_num', 'unqualified_num', 'production_address', 'food_product_time', 'food_model', 'check_flag', 'sampling_name', 'sampling_province', 'check_projiect']

with open(file_name, 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(total_info)
print('数据写入完成!')

糕点-不合格共有2478个食品结果
开始抓取数据.....
已完成20个食品的数据抓取
已完成40个食品的数据抓取
已完成60个食品的数据抓取
已完成80个食品的数据抓取
已完成100个食品的数据抓取
已完成120个食品的数据抓取
已完成140个食品的数据抓取
已完成160个食品的数据抓取
已完成180个食品的数据抓取
已完成200个食品的数据抓取
已完成220个食品的数据抓取
已完成240个食品的数据抓取
已完成260个食品的数据抓取
已完成280个食品的数据抓取
已完成300个食品的数据抓取
已完成320个食品的数据抓取
已完成340个食品的数据抓取
已完成360个食品的数据抓取
已完成380个食品的数据抓取
已完成400个食品的数据抓取
已完成420个食品的数据抓取
已完成440个食品的数据抓取
已完成460个食品的数据抓取
已完成480个食品的数据抓取
已完成500个食品的数据抓取
已完成520个食品的数据抓取
已完成540个食品的数据抓取
已完成560个食品的数据抓取
已完成580个食品的数据抓取
已完成600个食品的数据抓取
已完成620个食品的数据抓取
已完成640个食品的数据抓取
已完成660个食品的数据抓取
[INFO ERROR] food_id为7d53a250-20d0-406a-be02-8473df1d30dd的食品无法抓取
已完成680个食品的数据抓取
[INFO ERROR] food_id为7d53a250-20d0-406a-be02-8473df1d30dd的食品无法抓取
已完成700个食品的数据抓取
[INFO ERROR] food_id为7d53a250-20d0-406a-be02-8473df1d30dd的食品无法抓取
已完成720个食品的数据抓取


ConnectionError: HTTPConnectionPool(host='www.foods12331.cn', port=80): Max retries exceeded with url: /food/detail/getResult.json (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x7f6beb48b898>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [6]:
food_type = '糖果制品'
check_flag = '不合格'
file_name = food_type+'-'+check_flag+'.csv'
if check_flag == '合格':
    order_by = 1
else:
    order_by = 0
    
info = {}
info['food_type'] = food_type.encode('UTF-8')
info['check_flag'] = check_flag.encode('UTF-8')
info = urllib.parse.urlencode(info)
info = info.replace('&', '%22%2C%22').replace('=','%22%3A%22')+'%22%2C%22'    


total_info = []
session = requests.Session()
session.cookies = cookielib.LWPCookieJar(filename='sacspider')
try:
    session.cookies.load(ignore_discard=True)
except:
    print('cookie 文件未能加载')

    
url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
headers = {
        'Accept':'application/json',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        'Host':'www.foods12331.cn',
        'Origin':'http://www.foods12331.cn',
        'proxy-connection': "keep-alive",
        'cache-control': "no-cache",
        'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
    }
    
payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22'.format(order_by)+'pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(0)

response = session.post(url=url, data=payload, headers=headers)
total_food = response.json()['resultData']['total']
        
print(food_type+'-'+check_flag+'共有{}个食品结果'.format(total_food))
food_page = total_food // 20
print('开始抓取数据.....')

for i in range(0,food_page,1):
#     print(i)
    url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
    headers = {
        'Accept':'application/json',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        'Host':'www.foods12331.cn',
        'Origin':'http://www.foods12331.cn',
        'proxy-connection': "keep-alive",
        'cache-control': "no-cache",
        'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
    }
#     payload = "filters=%7B%22food_type%22%3A%22%E9%A5%AE%E6%96%99%22%2C%22check_flag%22%3A%22%E5%90%88%E6%A0%BC%22%2C%22order_by%22%3A%221%22%2C%22pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D".format(i)
    

    payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22'.format(order_by)+'pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(i)

    response = session.post(url=url, data=payload, headers=headers)
    if response.status_code // 100 != 2:
        print('[INFO ERROR] 第'+str(i)+'页内容无法爬取')
        continue
    for ele in response.json()['resultData']['items']:
        params = {}
        food_id = ele['id']
        food_name = ele['food_name']
        production_name = ele['production_name']
        check_num = ele['check_num']
        qualified_num = ele['qualified_num']
        unqualified_num = ele['unqualified_num']
        params['food_name'] = ele['food_name'].encode('UTF-8')
        params['production_name'] = ele['production_name'].encode('UTF-8')
        params['food_model'] = ele['food_model'].encode('UTF-8')
        payload = urllib.parse.urlencode(params)
        url = 'http://www.foods12331.cn/food/detail/getResult.json'
        headers = {
            'Accept':'*/*',
            'Accept-Encoding':'gzip, deflate',
            'Accept-Language':'zh-CN,zh;q=0.8',
            'Connection':'keep-alive',
            'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
            'Host':'www.foods12331.cn',
            'Origin':'http://www.foods12331.cn',
            'proxy-connection': "keep-alive",
            'cache-control': "no-cache",
            'Referer':'http://www.foods12331.cn/web/list.jsp?foodId={}&food_type=%E9%A5%AE%E6%96%99&food_condition='.format(food_id),
            'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'X-Requested-With':'XMLHttpRequest'
        }
        sleep_time = random.randint(2,4)
        time.sleep(sleep_time)
        response = session.post(url=url, data=payload, headers=headers)
        if response.status_code // 100 != 2:
            print('[INFO ERROR] food_id为'+food_id+'的食品无法抓取')
            continue
        for ele in response.json()['resultData']['foods']:
            food_dict = {}
            production_address = ele['production_adress']
            food_product_time = ele['food_product_time']
            food_model = ele['food_model']
            check_flag = ele['check_flag']
            if order_by == 0:
                if ele['unqualified_reason']:
                    check_flag = check_flag + " " + ele['unqualified_reason']
            sampling_name = ele['sampling_name']
            sampling_province = ele['sampling_province']
            check_projiect = ele['check_projiect']

            food_dict = {
                'food_name':food_name,
                'production_name':production_name,
                'check_num':check_num,
                'qualified_num':qualified_num,
                'unqualified_num':unqualified_num,
                'production_address':production_address,
                'food_product_time':food_product_time,
                'food_model':food_model,
                'check_flag':check_flag,
                'sampling_name':sampling_name,
                'sampling_province':sampling_province,
                'check_projiect':check_projiect
                }
            total_info.append(food_dict)
#             print(food_dict)
#         sleep_time = random.randint(1, 4)
#         time.sleep(sleep_time)
    session.cookies.save()
    print(u'已完成'+ str((i+1)*20) +u'个食品的数据抓取')


# 最后一页page_size不是20时的数据抓取
remain_page_size = total_food % 20
remain_page = total_food // 20

print('抓取最后{}个食品的数据......'.format(remain_page_size))

url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
headers = {
    'Accept':'application/json',
    'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.8',
    'Connection':'keep-alive',
    'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
    'Host':'www.foods12331.cn',
    'Origin':'http://www.foods12331.cn',
    'proxy-connection': "keep-alive",
    'cache-control': "no-cache",
    'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'X-Requested-With':'XMLHttpRequest'
}


payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22pageNo%22%3A{}%2C%22pageSize%22%3A{}%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(order_by,remain_page,remain_page_size)

response = session.post(url=url, data=payload, headers=headers)

flag = True

if response.status_code // 100 != 2:
    print('[INFO ERROR] 第'+str(i)+'页内容无法爬取')
    flag = False
if flag:
    for ele in response.json()['resultData']['items']:
        params = {}
        food_id = ele['id']
        food_name = ele['food_name']
        production_name = ele['production_name']
        check_num = ele['check_num']
        qualified_num = ele['qualified_num']
        unqualified_num = ele['unqualified_num']
        params['food_name'] = ele['food_name'].encode('UTF-8')
        params['production_name'] = ele['production_name'].encode('UTF-8')
        params['food_model'] = ele['food_model'].encode('UTF-8')
        payload = urllib.parse.urlencode(params)
        url = 'http://www.foods12331.cn/food/detail/getResult.json'
        headers = {
                'Accept':'*/*',
                'Accept-Encoding':'gzip, deflate',
                'Accept-Language':'zh-CN,zh;q=0.8',
                'Connection':'keep-alive',
                'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
                'Host':'www.foods12331.cn',
                'Origin':'http://www.foods12331.cn',
                'proxy-connection': "keep-alive",
                'cache-control': "no-cache",
                'Referer':'http://www.foods12331.cn/web/list.jsp?foodId={}&food_type=%E9%A5%AE%E6%96%99&food_condition='.format(food_id),
                'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
                'X-Requested-With':'XMLHttpRequest'
            }
        time.sleep(2)
        response = session.post(url=url, data=payload, headers=headers)
        if response.status_code // 100 != 2:
            print('[INFO ERROR] food_id为'+food_id+'的食品无法抓取')
            continue
        for ele in response.json()['resultData']['foods']:
            food_dict = {}
            production_address = ele['production_adress']
            food_product_time = ele['food_product_time']
            food_model = ele['food_model']
            check_flag = ele['check_flag']
            if order_by == 0:
                if ele['unqualified_reason']:
                    check_flag = check_flag + " " + ele['unqualified_reason']
            sampling_name = ele['sampling_name']
            sampling_province = ele['sampling_province']
            check_projiect = ele['check_projiect']
            food_dict = {
                'food_name':food_name,
                'production_name':production_name,
                'check_num':check_num,
                'qualified_num':qualified_num,
                'unqualified_num':unqualified_num,
                'production_address':production_address,
                'food_product_time':food_product_time,
                'food_model':food_model,
                'check_flag':check_flag,
                'sampling_name':sampling_name,
                'sampling_province':sampling_province,
                'check_projiect':check_projiect
                }
            total_info.append(food_dict)
    
print('数据抓取完成!\n开始写入数据.....')
    
headers = ['food_name', 'production_name', 'check_num', 'qualified_num', 'unqualified_num', 'production_address', 'food_product_time', 'food_model', 'check_flag', 'sampling_name', 'sampling_province', 'check_projiect']

with open(file_name, 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(total_info)
print('数据写入完成!')

糖果制品-不合格共有88个食品结果
开始抓取数据.....
已完成20个食品的数据抓取
已完成40个食品的数据抓取
已完成60个食品的数据抓取
已完成80个食品的数据抓取
抓取最后8个食品的数据......
数据抓取完成!
开始写入数据.....
数据写入完成!


In [15]:
# headers = ['food_name', 'production_name', 'check_num', 'qualified_num', 'unqualified_num', 'production_address', 'food_product_time', 'food_model', 'check_flag', 'sampling_name', 'sampling_province', 'check_projiect']
# with open('sac_info_full.csv', 'w') as f:
#     f_csv = csv.DictWriter(f, headers)
#     f_csv.writeheader()
#     f_csv.writerows(total_info)
    
headers = ['food_name', 'production_name', 'check_num', 'qualified_num', 'unqualified_num', 'production_address', 'food_product_time', 'food_model', 'check_flag', 'sampling_name', 'sampling_province', 'check_projiect']
with open('肉制品-不合格.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(total_info)